In [146]:
#!pip install display_xml

Importamos las librerías necesarias

In [199]:
import requests
import xml.dom.minidom
import xml.etree.ElementTree as ET
import re
import pandas as pd

Definimos funciones que nos servirán más adelante

In [190]:
def getRegexText(regex, labelText): #Para obtener los textos de un string que hagan match con la expresión regular
  matches =  re.finditer(regex, labelText, re.MULTILINE) # regex multilinea
  for matchNum, match in enumerate(matches, start=1):
    texto = match.group()
    return texto

def get_namespace(element): # Para obtener el namespace
  m = re.match('\{.*\}', element.tag)
  return m.group(0) if m else ''

def getTagName(tag, namespace):
  return namespace + tag

Hacemos la petición de descarga para el archivo solicitado y lo guardamos de manera local como xml

In [148]:
url = 'https://raw.githubusercontent.com/mcd-unison/ing-caract/main/ejemplos/integracion/ejemplos/wikipedia-poetas.xml'

response = requests.get(url)
with open('poetas.xml', 'wb') as file:
    file.write(response.content)


utilizamos las funciones de la biblioteca xml.dom.minidom para mostrar la información del xml estilo que es más fácil ára visualizar su estructura

In [ ]:
dom = xml.dom.minidom.parse('poetas.xml') # or xml.dom.minidom.parseString(xml_string)
pretty_xml_as_string = dom.toprettyxml()
print(pretty_xml_as_string)

Como podemos observar, se genera todo un árbol que presenta cierta estructura, aquí vemos que tenemos al mismo nivel la etiqueta siteinfo y page, de las cuales siteinfo no la necesitamos y también se tiene que por cada etiqueta page está contenida la información de un poeta argentino.

Extraemos la información del xml con las funciones de ElementTree

In [166]:
tree = ET.parse('poetas.xml')
root = tree.getroot()

Obtenemos el namespace del xml

In [244]:
namespace = get_namespace(tree.getroot()) # Obtenemos el namespace del xml
print(namespace)

{http://www.mediawiki.org/xml/export-0.10/}


In [241]:
df = pd.DataFrame(columns=['idAutor', 'Autor', 'IdObra', 'Obra'])

In [242]:
regexBiografia = r"\={2,3}\sObras?\s\={2,3}\s(\n.|.)*[^\S]{2}"
regexEBio = r"^\*\s.*"

i = 0
j = 0

for page in root.findall(getTagName('page', namespace)):
    autor = page.find(getTagName('title', namespace))
    title =  page.find(getTagName('revision', namespace))
    texto = title.find(getTagName('text', namespace))
    Biografias = getRegexText(regexBiografia, texto.text)
    if Biografias != None:
        matches = re.finditer(regexEBio, Biografias, re.MULTILINE)
        for matchNum, match in enumerate(matches, start=1):
            b = match.group()
            df.loc[j] = [i, autor.text, matchNum, b]
            j += 1
    i += 1


In [243]:
print(df)

      idAutor                      Autor  IdObra  \
0           0  Julia Morilla de Campbell       1   
1           0  Julia Morilla de Campbell       2   
2           0  Julia Morilla de Campbell       3   
3           0  Julia Morilla de Campbell       4   
4           0  Julia Morilla de Campbell       5   
...       ...                        ...     ...   
1831      632                 Ireneo Paz       5   
1832      632                 Ireneo Paz       6   
1833      632                 Ireneo Paz       7   
1834      634            Humberto Tejera       1   
1835      634            Humberto Tejera       2   

                                                   Obra  
0     * 1976: ''Crines al viento''.<ref>[http://abr....  
1     * 1976: ''Adiós Canopus: viaje por la Mesopota...  
2     * 1978: ''Viajando con el calendario escolar: ...  
3     * 1978: ''El rey: aventuras de un cóndor neuqu...  
4     * 1976: ''Sombrero negro, vida y aventuras en ...  
...                        

In [181]:
regex = r"\={2,3}\sObras\s\={2,3}\s(\n.|.)*[^\S]{2}"
matches =  re.finditer(regex, te, re.MULTILINE)
for matchNum, match in enumerate(matches, start=1):
    a = match.group()
    print(a)

== Obras ==

* 1976: ''Crines al viento''.<ref>[http://abr.uns.edu.ar/cgi-bin/opacmarc/wxis?IsisScript=opac/xis/opac.xis&db=abr&task=BIB-H-SEARCH&index=NAME&query=^aMorilla+de+Campbell,+Julia. Asociación Bernardino Rivadavia] (en el sitio de esta biblioteca popular se lo menciona como ''Crímenes al viento'' [sic, por ''Crines al viento'']).</ref>
* 1976: ''Adiós Canopus: viaje por la Mesopotamia hasta la selva misionera''.
* 1978: ''Viajando con el calendario escolar: poesías de clases alusivas''.
* 1978: ''El rey: aventuras de un cóndor neuquino''.
* 1976: ''Sombrero negro, vida y aventuras en la selva chaqueña''. Buenos Aires: Plus Ultra.




In [162]:
regex = r"^\*\s.*"
matches = re.finditer(regex, a, re.MULTILINE)
for matchNum, match in enumerate(matches, start=1):
    b = match.group()
    print(b)

* Bertero, Gloria de: ''¿Quién es ella en Santa Fe?'' (biografías de mujeres santafesinas). Rosario: Vinciguerra (Testimonios), 2001, 304&nbsp;págs. ISBN 9789508436948 y 950-43-6663-5.


In [231]:
df.to_parquet('data.parquet')

In [234]:
parquet_file = 'data.parquet' 
pd.read_parquet(parquet_file, engine='pyarrow')

,Autor,Obra,id
0,Julia Morilla de Campbell,* 1976: ''Crines al viento''.<ref>[http://abr....,1
1,Julia Morilla de Campbell,* 1976: ''Adiós Canopus: viaje por la Mesopota...,2
2,Julia Morilla de Campbell,* 1978: ''Viajando con el calendario escolar: ...,3
3,Julia Morilla de Campbell,* 1978: ''El rey: aventuras de un cóndor neuqu...,4
4,Julia Morilla de Campbell,"* 1976: ''Sombrero negro, vida y aventuras en ...",5
...,...,...,...
1831,Ireneo Paz,"* ''Leyendas históricas de la Independencia'',...",5
1832,Ireneo Paz,"* ''Vida y aventuras de Joaquín Murrieta, famo...",6
1833,Ireneo Paz,"* ''Leyendas históricas'', [[1914]]",7
1834,Humberto Tejera,* ''Los Gómez y el poder judicial de Venezuela...,1
